## Leiden Clustering

In [5]:
# Importing libraries for single-cell RNA analysis and probabilistic modeling.
import scanpy as sc
import scvi


In [6]:
# Defining the base path to the directory containing single-cell RNA-seq data.
# Make sure to update this path according to your file system.
base_data_path = "/Users/klemkelab/PDAC_scRNAseq_PLAUR_GSE205013/data"


In [7]:
# Reading the combined single-cell RNA-seq data stored in an .h5ad file using Scanpy.
# The file path is constructed using the base data path.
adata = sc.read_h5ad(f'{base_data_path}/data_combined.h5ad')


In [9]:
# Grouping the observation metadata in the AnnData object by 'Sample' and counting the number of observations per sample.
# The 'observed' parameter is set to False to retain the current behavior and silence the warning.
adata.obs.groupby('Sample', observed=False).count()


,doublet,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_ribo,pct_counts_ribo,n_genes
Sample,,,,,,,,
Treated,50020,50020,50020,50020,50020,50020,50020,50020
Untreated,23560,23560,23560,23560,23560,23560,23560,23560


In [10]:
# Creating a new layer 'counts' in the AnnData object by making a copy of the original data matrix (adata.X) to preserve raw count data.
adata.layers['counts'] = adata.X.copy()


In [11]:
# Normalizing the total counts per cell so that each cell has a total count of 10,000. This makes the data comparable across cells.
sc.pp.normalize_total(adata, target_sum=1e4)

# Applying a logarithmic transformation (log1p) to the data to stabilize variance and make the data more normally distributed.
sc.pp.log1p(adata)

# Storing the preprocessed (log-normalized) data in the raw attribute of the AnnData object, preserving it for future use.
adata.raw = adata


In [12]:
adata

AnnData object with n_obs × n_vars = 73580 × 26811
    obs: 'Sample', 'doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'n_genes'
    var: 'n_cells'
    uns: 'log1p'
    layers: 'counts'

In [16]:
# Setup the AnnData object for use with scVI, specifying the layer where raw counts are stored ('counts').
scvi.model.SCVI.setup_anndata(adata, layer='counts')

# Initialize the SCVI model with the corrected AnnData object.
model = scvi.model.SCVI(adata)


In [17]:
import torch

if torch.backends.mps.is_available():
    print("MPS backend is available for training on GPU!")
else:
    print("MPS backend is not available. Training will default to CPU.")


MPS backend is available for training on GPU!


In [20]:
import torch

# Check if MPS is available and set the device to MPS, otherwise use CPU
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Move the SCVI model to the MPS device
model.to_device(device)

# Train the SCVI model using the SCVI's internal .train() method
model.train()


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/klemkelab/PDAC_scRNAseq_PLAUR_GSE205013/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/Users/klemkelab/PDAC_scRNAseq_PLAUR_GSE205013/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training:   0%|          | 0/109 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=109` reached.


In [21]:
# Save the trained SCVI model to the specified directory.
# Note: Delete any previously saved model folder before running this code to avoid errors.
model.save(f'{base_data_path}/model.model')


In [24]:
# Extracting the latent representation (low-dimensional embeddings) from the trained SCVI model.
# Storing it in the AnnData object's 'obsm' attribute under 'X_scVI' for downstream analysis, such as clustering or visualization.
adata.obsm['X_scVI'] = model.get_latent_representation()


In [22]:
# Extracting the normalized gene expression from the SCVI model, with a specified library size of 10,000,
# and storing it in the AnnData object's layers under 'scvi_normalized' for further analysis or downstream tasks.
adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size=1e4)


In [26]:
# Computing the neighborhood graph of cells using the latent representation 'X_scVI'.
# This graph is used for clustering and other downstream analysis.
sc.pp.neighbors(adata, use_rep='X_scVI')

# Performing Leiden clustering with the 'igraph' backend and recommended parameters to avoid future warnings.
# 'directed=False' is set to work with igraph's implementation.
sc.tl.leiden(adata, resolution=0.1, flavor="igraph", directed=False, n_iterations=2)


In [27]:
# Saving the AnnData object, including all integrated data and results (such as Leiden clusters), 
# to an .h5ad file for later use or analysis.
adata.write_h5ad(f'{base_data_path}/integrated_0.1.h5ad')
